In [9]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8

In [2]:
# Relative path to data files
DATA_DIR = '../data/'

In [3]:
# List data files in DATA_DIR
files = sorted(
    [file for file in os.listdir(DATA_DIR) if file[-4:] == '.csv'])
files

['AlpineMedowsWA.csv',
 'BigMedowNV.csv',
 'CO2stations.csv',
 'COstations2.csv',
 'ID3stations.csv',
 'OR3stations.csv',
 'SForkShieldsMT.csv',
 'UT2stations.csv',
 'WA4stations.csv',
 'WY2stations.csv',
 'blue_mt_springOR.csv',
 'deerParkWY_2007_2016.csv',
 'elkcabinNM.csv',
 'eugeneOR_2007_2016.csv',
 'fairbanksAK_2007_2016.csv',
 'grandviewAK.csv',
 'hyndmanID.csv',
 'waterholeWA_2007_2016.csv',
 'wildcatAZ.csv']

In [20]:
# Read data
data_daily = pd.read_csv(DATA_DIR + files[0])

# Convert DATE to datetime (instead of string)
data_daily['DATE']= pd.to_datetime(data_daily['DATE'])
data_daily.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,TAVG,TMAX,TMIN
0,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-01,38.1,2.3,5.3,0.0
1,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-02,66.0,4.4,6.5,2.5
2,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-03,61.0,1.2,2.5,-0.6
3,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-04,30.5,-1.9,-0.2,-3.4
4,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-05,0.0,-1.6,0.3,-4.2


In [21]:
# Extract YEAR and MONTH from DATE
data_daily['YEAR'] = data_daily.DATE.apply(lambda x: x.year)
data_daily['MONTH'] = data_daily.DATE.apply(lambda x: x.month)
data_daily.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,TAVG,TMAX,TMIN,YEAR,MONTH
0,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-01,38.1,2.3,5.3,0.0,2007,1
1,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-02,66.0,4.4,6.5,2.5,2007,1
2,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-03,61.0,1.2,2.5,-0.6,2007,1
3,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-04,30.5,-1.9,-0.2,-3.4,2007,1
4,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-05,0.0,-1.6,0.3,-4.2,2007,1


In [24]:
# Aggregate by month: get total monthly precipitation, and monthly 
# avg, min, and max temps
data_monthly = data_daily.groupby(['YEAR', 'MONTH']).agg(
    {'YEAR': min, 
     'MONTH': min, 
     'PRCP': sum, 
     'TAVG': np.mean, 
     'TMAX': max, 
     'TMIN': min})
data_monthly = data_monthly.reset_index(drop=True)
data_monthly.head()

,YEAR,MONTH,PRCP,TAVG,TMAX,TMIN
0,2007,1,579.1,3.724619e-16,11.5,-10.7
1,2007,2,454.5,1.700000e+00,10.5,-6.1
2,2007,3,721.3,2.625806e+00,14.9,-7.3
3,2007,4,274.4,3.526667e+00,19.9,-4.1
4,2007,5,93.8,7.261290e+00,25.0,0.0


In [29]:
data_monthly['YEAR_MONTH'] = data_monthly.apply(
    lambda row: pd.to_datetime(
        str(int(row['YEAR'])) + '-' + str(int(row['MONTH']))), 
    axis=1)
data_monthly.head()

,YEAR,MONTH,PRCP,TAVG,TMAX,TMIN,YEAR_MONTH
0,2007,1,579.1,3.724619e-16,11.5,-10.7,2007-01-01
1,2007,2,454.5,1.700000e+00,10.5,-6.1,2007-02-01
2,2007,3,721.3,2.625806e+00,14.9,-7.3,2007-03-01
3,2007,4,274.4,3.526667e+00,19.9,-4.1,2007-04-01
4,2007,5,93.8,7.261290e+00,25.0,0.0,2007-05-01


In [38]:
def aggregate_daily_file_to_monthly(daily_data):
    if 'TAVG' not in list(data_daily):
        print('"TAVG" field not found in %s.  Not using data.' % filepath)
    data_daily['DATE']= pd.to_datetime(data_daily['DATE'])
    data_daily['YEAR'] = data_daily.DATE.apply(lambda x: x.year)
    data_daily['MONTH'] = data_daily.DATE.apply(lambda x: x.month)
    data_monthly = data_daily.groupby(['YEAR', 'MONTH'])\
        .agg({'YEAR': min, 
              'MONTH': min,
              'PRCP': sum, 
              'TAVG': np.mean, 
              'TMAX': max, 
              'TMIN': min})
    data_monthly['YEAR_MONTH'] = data_monthly.apply(
        lambda row: pd.to_datetime(
            str(int(row['YEAR'])) + '-' + str(int(row['MONTH']))), 
        axis=1)
    data_monthly['TAVG'] = np.round(data_monthly['TAVG'], 1)
    return data_monthly.reset_index(drop=True)

In [39]:
data_daily = pd.read_csv(DATA_DIR + files[0])
data_monthly = aggregate_daily_file_to_monthly(data_daily)

In [40]:
data_monthly.head()

,YEAR,MONTH,PRCP,TAVG,TMAX,TMIN,YEAR_MONTH
0,2007,1,579.1,0.0,11.5,-10.7,2007-01-01
1,2007,2,454.5,1.7,10.5,-6.1,2007-02-01
2,2007,3,721.3,2.6,14.9,-7.3,2007-03-01
3,2007,4,274.4,3.5,19.9,-4.1,2007-04-01
4,2007,5,93.8,7.3,25.0,0.0,2007-05-01


In [41]:
data_daily['YEAR_MONTH'] = data_daily.apply(
        lambda row: pd.to_datetime(
            str(int(row['YEAR'])) + '-' + str(int(row['MONTH']))), 
        axis=1)
data_daily.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,TAVG,TMAX,TMIN,YEAR,MONTH,YEAR_MONTH
0,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-01,38.1,2.3,5.3,0.0,2007,1,2007-01-01
1,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-02,66.0,4.4,6.5,2.5,2007,1,2007-01-01
2,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-03,61.0,1.2,2.5,-0.6,2007,1,2007-01-01
3,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-04,30.5,-1.9,-0.2,-3.4,2007,1,2007-01-01
4,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-05,0.0,-1.6,0.3,-4.2,2007,1,2007-01-01


In [42]:
# Merge monthly with daily
data = data_daily.merge(data_monthly, how='left', on='YEAR_MONTH')
data.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP_x,TAVG_x,TMAX_x,TMIN_x,YEAR_x,MONTH_x,YEAR_MONTH,YEAR_y,MONTH_y,PRCP_y,TAVG_y,TMAX_y,TMIN_y
0,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-01,38.1,2.3,5.3,0.0,2007,1,2007-01-01,2007,1,579.1,0.0,11.5,-10.7
1,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-02,66.0,4.4,6.5,2.5,2007,1,2007-01-01,2007,1,579.1,0.0,11.5,-10.7
2,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-03,61.0,1.2,2.5,-0.6,2007,1,2007-01-01,2007,1,579.1,0.0,11.5,-10.7
3,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-04,30.5,-1.9,-0.2,-3.4,2007,1,2007-01-01,2007,1,579.1,0.0,11.5,-10.7
4,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-05,0.0,-1.6,0.3,-4.2,2007,1,2007-01-01,2007,1,579.1,0.0,11.5,-10.7


In [43]:
drops = ['YEAR_y', 'MONTH_y']
name_map = {'STATION': 'station',
            'NAME': 'name',
            'LATITUDE': 'lat',
            'LONGITUDE': 'lon',
            'ELEVATION': 'elev',
            'DATE': 'date',
            'PRCP_x': 'precip_daily',
            'TAVG_x': 'avg_temp_daily',
            'TMAX_x': 'max_temp_daily',
            'TMIN_x': 'min_temp_daily',
            'YEAR_x': 'year',
            'MONTH_x': 'month',
            'YEAR_MONTH': 'year_month',
            'PRCP_y': 'precip_monthly',
            'TAVG_y': 'avg_temp_monthly',
            'TMAX_y': 'max_temp_monthly',
            'TMIN_y': 'min_temp_monthly'}
data = data.drop(drops, axis=1)
data = data.rename(columns=name_map)
data.head()

,station,name,lat,lon,elev,date,precip_daily,avg_temp_daily,max_temp_daily,min_temp_daily,year,month,year_month,precip_monthly,avg_temp_monthly,max_temp_monthly,min_temp_monthly
0,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-01,38.1,2.3,5.3,0.0,2007,1,2007-01-01,579.1,0.0,11.5,-10.7
1,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-02,66.0,4.4,6.5,2.5,2007,1,2007-01-01,579.1,0.0,11.5,-10.7
2,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-03,61.0,1.2,2.5,-0.6,2007,1,2007-01-01,579.1,0.0,11.5,-10.7
3,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-04,30.5,-1.9,-0.2,-3.4,2007,1,2007-01-01,579.1,0.0,11.5,-10.7
4,USS0021B48S,"ALPINE MEADOWS, WA US",47.78,-121.7,1066.8,2007-01-05,0.0,-1.6,0.3,-4.2,2007,1,2007-01-01,579.1,0.0,11.5,-10.7


# Create variables based on daily data
### Early Cold Snap
For 15 Oct - 30 Nov: ≥4 consecutive days of ≤ -20C

In [44]:
def split_into_annual(data):
    years = data.year.unique()
    yearly_data_sets = []
    for year in years:
        yearly_data = data.loc[data.year == year, :]
        yearly_data_sets.append(yearly_data)
    return yearly_data_sets

In [46]:
len(data.year.unique())

10

In [48]:
yearly_data_sets = split_into_annual(data)
len(yearly_data_sets)

10

In [66]:
def has_early_cold_snap(data, breached_days=4, threshold=-20):
    if len(data.year.unique()) > 1:
        print('has_early_cold_snap() requires a single year of data')
        return None
    year = data.year.unique()[0]
    start_date = pd.to_datetime('%s-10-15' % year)
    end_date = pd.to_datetime('%s-11-30' % year)
    print(start_date, end_date)
    date_range_of_interest = data.loc[
        ((data.date >= start_date) & (data.date <= end_date)), :]
    if date_range_of_interest.min_temp_daily.min() > threshold:
        return False
    threshold_breached = (date_range_of_interest['min_temp_daily'] 
                          <= threshold)
    return threshold_breached.cumsum().max() >= breached_days

# Test
has_early_cold_snap(yearly_data_sets[0])

2007-10-15 00:00:00 2007-11-30 00:00:00


False